In [ ]:
from bayes_opt import BayesianOptimization
import subprocess
from time import time_ns

In [ ]:
def run_spind(CHUNK_SIZE, SORT_SIZE, MERGE_SIZE, VALIDATION_SIZE):
    start_time = time_ns()
    
    # execute spind using the provided parameters
    output = subprocess.check_output(['java', '-jar', 'spind.jar', str(round(CHUNK_SIZE)), str(round(SORT_SIZE)), str(round(MERGE_SIZE)), str(round(VALIDATION_SIZE))])

    # we want to minimize time -> return negative time since the optimizer is maximizing
    # return time in seconds with millisecond resolution
    return -(((time_ns() - start_time)//1_000_000)/1_000)

In [ ]:
parameter_bounds = {
    'CHUNK_SIZE': (10_000, 100_000_000),
    'SORT_SIZE': (10_000, 5_000_000),
    'MERGE_SIZE': (2, 1_000),
    'VALIDATION_SIZE': (1, 1_000_000)
}

In [ ]:
optimizer = BayesianOptimization(
    f=run_spind,
    pbounds=parameter_bounds,
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

In [ ]:
optimizer.maximize(20, 65)

In [ ]:
import pandas as pd
import numpy as np

OPTIMIZATION_NAME = 'TPCH-1'

data = []
for r in optimizer.res:
    t = r['params']
    t['time'] = r['target']
    data.append(t)
df = pd.DataFrame(data)
df.values[:,:-1] = np.round(df.values[:,:-1])
df.to_csv(f'data/{OPTIMIZATION_NAME}.csv')